In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-cleaned-datasets/test_data.parquet
/kaggle/input/amex-cleaned-datasets/train_data_cleaned.parquet
/kaggle/input/amex-cleaned-datasets/events_train.parquet
/kaggle/input/amex-cleaned-datasets/train_plus_all_features.parquet
/kaggle/input/amex-cleaned-datasets/events_features_test.parquet
/kaggle/input/amex-cleaned-datasets/events_features_train.parquet
/kaggle/input/amex-cleaned-datasets/add_trans.parquet
/kaggle/input/amex-cleaned-datasets/offer_clean.parquet
/kaggle/input/amex-cleaned-datasets/id3_train_extended.parquet
/kaggle/input/amex-cleaned-datasets/firstsubmission.ipynb
/kaggle/input/amex-cleaned-datasets/df_train1.parquet
/kaggle/input/amex-cleaned-datasets/add_event_cleaned.parquet
/kaggle/input/amex-cleaned-datasets/events_test.parquet
/kaggle/input/amex-cleaned-datasets/id3_test_extended.parquet
/kaggle/input/amex-cleaned-datasets/test_data_cleaned.parquet
/kaggle/input/amex-cleaned-datasets/test_plus_all_features.parquet
/kaggle/input/amex-cleaned-datase

In [22]:
fml=pd.read_parquet('/kaggle/input/amex-cleaned-datasets/train_data_cleaned.parquet')
events=pd.read_parquet('/kaggle/input/amex-cleaned-datasets/add_event_cleaned.parquet')
trans=pd.read_parquet('/kaggle/input/amex-cleaned-datasets/add_trans.parquet')

In [23]:
common_id3s = set(fml['id3']).intersection(events['id3'])
common_pairs = events[events['id3'].isin(common_id3s)][['id3', 'id2']].drop_duplicates()
common_pairs

,id3,id2
0,618619,2431360
2,97193,2431360
3,654444,2431360
5,34229,2431360
6,428865,2202020
...,...,...
21457466,97646,2878066
21457467,92991,2878066
21457469,355406,2878066
21457470,146933,2878066


In [24]:

trans_overlap = trans[trans['id2'].isin(common_pairs['id2'])]
trans_overlap

,id2,f367,f368,f369,f370,f371,f372,id8,f374
0,2896709,15.60,PBR,D,2023-10-16,19:16:52,202310,59639998,DSE
2,2497175,13.99,PBR,D,2023-10-14,00:31:48,202310,59639998,DSE
3,2655364,15.14,PGC,D,2023-10-13,12:37:25,202310,59639998,DSE
6,2166784,13.99,PR,D,2023-10-06,16:27:55,202310,59639998,DSE
7,2487698,2.99,PR,D,2023-10-15,20:45:06,202310,59639998,DSE
...,...,...,...,...,...,...,...,...,...
6339454,2304917,150.00,PBR,D,2023-10-13,00:00:00,202310,59430301,SS
6339457,2145000,14.52,PPC,D,2023-10-25,00:00:00,202310,59430301,SS
6339458,2318854,11.26,PR,D,2023-10-18,00:00:00,202310,59430301,SS
6339459,2641784,91.58,OGC,D,2023-10-03,00:00:00,202310,59430301,SS


In [14]:
trans_overlap['f367'].skew()


173.6629819673037

In [25]:
trans_overlap.drop('f372', axis=1, inplace=True)
trans_overlap



/tmp/ipykernel_180/2459447374.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans_overlap.drop('f372', axis=1, inplace=True)


,id2,f367,f368,f369,f370,f371,id8,f374
0,2896709,15.60,PBR,D,2023-10-16,19:16:52,59639998,DSE
2,2497175,13.99,PBR,D,2023-10-14,00:31:48,59639998,DSE
3,2655364,15.14,PGC,D,2023-10-13,12:37:25,59639998,DSE
6,2166784,13.99,PR,D,2023-10-06,16:27:55,59639998,DSE
7,2487698,2.99,PR,D,2023-10-15,20:45:06,59639998,DSE
...,...,...,...,...,...,...,...,...
6339454,2304917,150.00,PBR,D,2023-10-13,00:00:00,59430301,SS
6339457,2145000,14.52,PPC,D,2023-10-25,00:00:00,59430301,SS
6339458,2318854,11.26,PR,D,2023-10-18,00:00:00,59430301,SS
6339459,2641784,91.58,OGC,D,2023-10-03,00:00:00,59430301,SS


In [26]:
# Check number of unique categories for each categorical feature
categorical_cols = ['f368', 'f369', 'f374', 'id8']

for col in categorical_cols:
    n_unique = trans_overlap[col].nunique()
    print(f"{col} has {n_unique} unique values")


f368 has 13 unique values
f369 has 2 unique values
f374 has 503 unique values
id8 has 4970 unique values


In [27]:
import pandas as pd

# Step 1: Combine f370 (date) and f371 (time) into full datetime
trans_overlap['txn_datetime'] = pd.to_datetime(
    trans_overlap['f370'].astype(str) + ' ' + trans_overlap['f371'].astype(str),
    errors='coerce'
)

# Step 2: Create row-level features from txn_datetime
trans_overlap['txn_hour'] = trans_overlap['txn_datetime'].dt.hour             # Hour of transaction
trans_overlap['txn_dow'] = trans_overlap['txn_datetime'].dt.dayofweek         # Day of week (0=Mon)
trans_overlap['txn_dom'] = trans_overlap['txn_datetime'].dt.day               # Day of month
trans_overlap['txn_is_weekend'] = (trans_overlap['txn_dow'] >= 5).astype(int) # Weekend flag

# Step 3: Aggregate per id2

# Custom safe_mode function
def safe_mode(series):
    mode = series.mode()
    return mode.iloc[0] if not mode.empty else None

# Group and aggregate
features_id2 = trans_overlap.groupby('id2').agg({
    'f367': ['median', 'std', 'max', 'sum', 'count'],
    'txn_datetime': ['min', 'max'],
    'txn_hour': [safe_mode, 'std'],
    'txn_dow': safe_mode,
    'txn_dom': safe_mode,
    'txn_is_weekend': 'mean',
    'f368': safe_mode,
    'f369': safe_mode,
    'f374': safe_mode,
    'id8': safe_mode
})

# Flatten MultiIndex columns
features_id2.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in features_id2.columns]
features_id2 = features_id2.reset_index()

# Rename columns
features_id2 = features_id2.rename(columns={
    'f367_median': 'txn_amt_median',
    'f367_std': 'txn_amt_std',
    'f367_max': 'txn_amt_max',
    'f367_sum': 'txn_amt_sum',
    'f367_count': 'txn_amt_count',
    'txn_datetime_min': 'first_txn_date',
    'txn_datetime_max': 'last_txn_date',
    'txn_hour_<lambda>': 'txn_hour_mode',
    'txn_hour_std': 'txn_hour_std',
    'txn_dow': 'day_of_week_mode',
    'txn_dom': 'day_of_month_mode',
    'txn_is_weekend_mean': 'weekend_txn_ratio',
    'f368': 'f368_mode',
    'f369': 'f369_mode',
    'f374': 'f374_mode',
    'id8': 'id8_mode'
})

# Step 4: Extra temporal features
features_id2['days_since_last_txn'] = (pd.Timestamp.now().normalize() - features_id2['last_txn_date']).dt.days
features_id2['txn_date_range'] = (features_id2['last_txn_date'] - features_id2['first_txn_date']).dt.days


/tmp/ipykernel_180/4212113585.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans_overlap['txn_datetime'] = pd.to_datetime(
/tmp/ipykernel_180/4212113585.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans_overlap['txn_hour'] = trans_overlap['txn_datetime'].dt.hour             # Hour of transaction
/tmp/ipykernel_180/4212113585.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [28]:
# Step 1: Group by id2 and compute nunique for categorical-like columns
extra_features_id2 = trans_overlap.groupby('id2').agg({
    'f368': 'nunique',
    'f374': 'nunique',
    'id8': 'nunique'
}).reset_index()

# Rename columns
extra_features_id2 = extra_features_id2.rename(columns={
    'f368': 'f368_nunique',
    'f374': 'f374_nunique',
    'id8': 'id8_nunique'
})

# Step 2: Count of each category in f369
f369_counts = pd.crosstab(trans_overlap['id2'], trans_overlap['f369']).reset_index()
f369_counts.columns.name = None
f369_counts = f369_counts.rename(columns=lambda x: f'f369_count_{x}' if x != 'id2' else x)

# Step 3: Merge both sets of features
final_extra_features_id2 = pd.merge(extra_features_id2, f369_counts, on='id2', how='left')


In [29]:
final_extra_features_id2

,id2,f368_nunique,f374_nunique,id8_nunique,f369_count_C,f369_count_D
0,2000010,1,6,7,0,12
1,2000022,1,1,1,0,1
2,2000023,2,8,15,2,30
3,2000025,1,6,8,0,10
4,2000032,1,1,1,0,1
...,...,...,...,...,...,...
113947,2897751,1,3,3,0,3
113948,2897759,1,3,4,0,7
113949,2897793,1,1,1,0,2
113950,2898158,1,1,1,0,1


In [30]:
features_id2

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id2,txn_amt_median,txn_amt_std,txn_amt_max,txn_amt_sum,txn_amt_count,first_txn_date,last_txn_date,txn_hour_safe_mode,txn_hour_std,txn_dow_safe_mode,txn_dom_safe_mode,weekend_txn_ratio,f368_safe_mode,f369_safe_mode,f374_safe_mode,id8_safe_mode,days_since_last_txn,txn_date_range
0,2000010,39.255,229.815205,625.85,1988.86,12,2023-10-03 10:03:30,2023-11-03 13:29:19,0,6.971805,4,3,0.333333,PR,D,BS,59420000,615,31
1,2000022,2284.650,NaN,2284.65,2284.65,1,2023-10-26 19:48:00,2023-10-26 19:48:00,19,NaN,3,26,0.000000,PR,D,ATS,45120100,623,0
2,2000023,13.310,25.327353,100.00,712.87,32,2023-10-02 00:00:00,2023-11-03 21:51:50,0,8.288887,4,3,0.093750,PPC,D,EP,58120600,615,32
3,2000025,73.395,82.155547,250.00,774.49,10,2023-10-11 00:02:00,2023-11-03 00:00:00,0,8.044322,2,1,0.100000,PR,D,GSS,55410000,616,22
4,2000032,124.030,NaN,124.03,124.03,1,2023-11-01 00:00:00,2023-11-01 00:00:00,0,NaN,2,1,0.000000,PGC,D,LS,59210102,618,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113947,2897751,116.480,87.215419,185.40,314.05,3,2023-10-09 10:29:00,2023-10-17 00:00:00,0,5.033223,0,9,0.000000,PGC,D,BSN,54219902,633,7
113948,2897759,18.920,13.417718,39.90,133.12,7,2023-10-11 00:00:00,2023-10-31 00:00:00,0,6.429101,4,28,0.285714,PBR,D,GS,54110100,619,20
113949,2897793,182.450,0.000000,182.45,364.90,2,2023-10-30 10:08:04,2023-11-03 14:43:17,10,2.828427,0,3,0.000000,PR,D,AARSN,79999916,615,4
113950,2898158,4.990,NaN,4.99,4.99,1,2023-10-18 22:39:38,2023-10-18 22:39:38,22,NaN,2,18,0.000000,PR,D,CSN,48990100,631,0


In [31]:
# Merge new features into existing features_id2
features_id2 = features_id2.merge(final_extra_features_id2, on='id2', how='left')
features_id2

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,id2,txn_amt_median,txn_amt_std,txn_amt_max,txn_amt_sum,txn_amt_count,first_txn_date,last_txn_date,txn_hour_safe_mode,txn_hour_std,...,f369_safe_mode,f374_safe_mode,id8_safe_mode,days_since_last_txn,txn_date_range,f368_nunique,f374_nunique,id8_nunique,f369_count_C,f369_count_D
0,2000010,39.255,229.815205,625.85,1988.86,12,2023-10-03 10:03:30,2023-11-03 13:29:19,0,6.971805,...,D,BS,59420000,615,31,1,6,7,0,12
1,2000022,2284.650,NaN,2284.65,2284.65,1,2023-10-26 19:48:00,2023-10-26 19:48:00,19,NaN,...,D,ATS,45120100,623,0,1,1,1,0,1
2,2000023,13.310,25.327353,100.00,712.87,32,2023-10-02 00:00:00,2023-11-03 21:51:50,0,8.288887,...,D,EP,58120600,615,32,2,8,15,2,30
3,2000025,73.395,82.155547,250.00,774.49,10,2023-10-11 00:02:00,2023-11-03 00:00:00,0,8.044322,...,D,GSS,55410000,616,22,1,6,8,0,10
4,2000032,124.030,NaN,124.03,124.03,1,2023-11-01 00:00:00,2023-11-01 00:00:00,0,NaN,...,D,LS,59210102,618,0,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113947,2897751,116.480,87.215419,185.40,314.05,3,2023-10-09 10:29:00,2023-10-17 00:00:00,0,5.033223,...,D,BSN,54219902,633,7,1,3,3,0,3
113948,2897759,18.920,13.417718,39.90,133.12,7,2023-10-11 00:00:00,2023-10-31 00:00:00,0,6.429101,...,D,GS,54110100,619,20,1,3,4,0,7
113949,2897793,182.450,0.000000,182.45,364.90,2,2023-10-30 10:08:04,2023-11-03 14:43:17,10,2.828427,...,D,AARSN,79999916,615,4,1,1,1,0,2
113950,2898158,4.990,NaN,4.99,4.99,1,2023-10-18 22:39:38,2023-10-18 22:39:38,22,NaN,...,D,CSN,48990100,631,0,1,1,1,0,1


In [32]:
numeric_cols = features_id2.select_dtypes(include='number').columns.drop('id2', errors='ignore')
skew_vals = features_id2[numeric_cols].skew().sort_values(ascending=False)

print("Skewness of numeric features:\n")
print(skew_vals)


Skewness of numeric features:

txn_amt_std            75.153456
txn_amt_median         71.958549
txn_amt_sum            53.198353
txn_amt_max            47.877710
f369_count_C           35.854474
f369_count_D           26.366981
txn_amt_count          25.913533
days_since_last_txn     2.743945
txn_hour_safe_mode      2.343970
f368_nunique            2.152408
weekend_txn_ratio       1.458619
id8_nunique             1.079385
f374_nunique            0.956594
txn_dom_safe_mode       0.709958
txn_dow_safe_mode       0.247560
txn_hour_std           -1.290251
txn_date_range         -1.640879
dtype: float64


In [33]:
# Step 0: Merge id2-level features into common_pairs
id3_id2_features = common_pairs.merge(features_id2, on='id2', how='left')

# Step 1: Define safe_mode again
def safe_mode(series):
    mode = series.mode()
    return mode.iloc[0] if not mode.empty else None

# Step 2: Define aggregation plan based on skewness
median_cols = [
    'txn_amt_std', 'txn_amt_median', 'txn_amt_sum', 'txn_amt_max',
    'f369_count_C', 'f369_count_D', 'txn_amt_count',
    'days_since_last_txn', 'txn_hour_safe_mode',
    'f368_nunique', 'weekend_txn_ratio', 'id8_nunique', 'f374_nunique'
]

mean_cols = [
    'txn_dom_safe_mode', 'txn_dow_safe_mode',
    'txn_hour_std', 'txn_date_range'
]

# Step 3: Build aggregation dictionary
agg_funcs = {}

for col in median_cols:
    agg_funcs[col] = 'median'

for col in mean_cols:
    agg_funcs[col] = 'mean'

# Step 4: Group by id3 and aggregate
features_id3 = id3_id2_features.groupby('id3').agg(agg_funcs).reset_index()


In [17]:
print(features_id3.columns.tolist())


['id3', 'txn_amt_std', 'txn_amt_median', 'txn_amt_sum', 'txn_amt_max', 'f369_count_C', 'f369_count_D', 'txn_amt_count', 'days_since_last_txn', 'txn_hour_safe_mode', 'f368_nunique', 'weekend_txn_ratio', 'id8_nunique', 'f374_nunique', 'txn_dom_safe_mode', 'txn_dow_safe_mode', 'txn_hour_std', 'txn_date_range']


In [27]:
# Check unique values in one of the mode columns
features_id3['txn_dow_safe_mode'].value_counts()


txn_dow_safe_mode
0.0    499
4.0    167
1.0     37
2.0     14
3.0      7
6.0      1
Name: count, dtype: int64

In [34]:
df_train=pd.read_parquet('/kaggle/input/amex-cleaned-datasets/train_fml.parquet')
df_train

,id1,id2,id3,id4,id5,f1,f2,f3,f4,f5,...,mode_imp_dayofweek,median_click_latency_sec,count_first_impressions,median_prev_clicks,median_imp_hour_clickrate,mean_imp_weekday_clickrate,mean_imp_month_clickrate,pct_tiles,pct_mobile_timeline,pct_offerstab
0,1366776_189706075_16-23_2023-11-02 22:22:00.042,1366776,189706075,2023-11-02 22:22:00.042,2023-11-02,1.0,0.0,0.0,0.0,0.0,...,2,3.350529,184,4.0,0.019492,0.023959,0.027089,0.341759,0.073363,0.584878
1,1366776_89227_16-23_2023-11-01 23:51:24.999,1366776,89227,2023-11-01 23:51:24.999,2023-11-01,1.0,0.0,0.0,0.0,0.0,...,2,3.274787,154,5.0,0.019492,0.024056,0.027089,0.326949,0.067324,0.605727
2,1366776_35046_16-23_2023-11-01 00:30:59.797,1366776,35046,2023-11-01 00:30:59.797,2023-11-01,1.0,0.0,0.0,0.0,0.0,...,2,3.079222,162,4.0,0.019492,0.023979,0.027089,0.340184,0.074060,0.585756
3,1366776_6275451_16-23_2023-11-02 22:21:32.261,1366776,6275451,2023-11-02 22:21:32.261,2023-11-02,1.0,0.0,0.0,0.0,0.0,...,2,4.002239,159,4.0,0.019492,0.024007,0.027089,0.348157,0.077781,0.574062
4,1366776_78053_16-23_2023-11-02 22:21:34.799,1366776,78053,2023-11-02 22:21:34.799,2023-11-02,1.0,0.0,0.0,0.0,0.0,...,2,3.235978,172,4.0,0.019492,0.024047,0.027089,0.346538,0.075809,0.577654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770159,1896641_87731_16-23_2023-11-02 08:14:21.524,1896641,87731,2023-11-02 08:14:21.524,2023-11-02,0.0,0.0,0.0,0.0,42.0,...,0,4.081338,1396,1.0,0.019409,0.019306,0.019039,0.346053,0.095872,0.558075
770160,1896641_505604_16-23_2023-11-02 08:14:24.458,1896641,505604,2023-11-02 08:14:24.458,2023-11-02,0.0,0.0,0.0,0.0,42.0,...,2,4.515435,1043,1.0,0.019409,0.019338,0.019033,0.358186,0.090802,0.551012
770161,1896641_25212_16-23_2023-11-02 08:14:25.748,1896641,25212,2023-11-02 08:14:25.748,2023-11-02,0.0,0.0,0.0,0.0,42.0,...,0,5.458744,993,1.0,0.019409,0.019331,0.019067,0.359830,0.087373,0.552797
770162,1900765_95157_16-23_2023-11-02 06:08:25.900,1900765,95157,2023-11-02 06:08:25.900,2023-11-02,0.0,0.0,0.0,0.0,0.0,...,2,3.340820,153,5.0,0.019492,0.024058,0.027089,0.331873,0.067070,0.601057


In [35]:
df_train = df_train.merge(features_id3, how='left', on='id3')
df_train

,id1,id2,id3,id4,id5,f1,f2,f3,f4,f5,...,days_since_last_txn,txn_hour_safe_mode,f368_nunique,weekend_txn_ratio,id8_nunique,f374_nunique,txn_dom_safe_mode,txn_dow_safe_mode,txn_hour_std,txn_date_range
0,1366776_189706075_16-23_2023-11-02 22:22:00.042,1366776,189706075,2023-11-02 22:22:00.042,2023-11-02,1.0,0.0,0.0,0.0,0.0,...,616.0,0.0,1.0,0.230769,14.0,10.0,10.616067,2.564816,7.267796,26.146657
1,1366776_89227_16-23_2023-11-01 23:51:24.999,1366776,89227,2023-11-01 23:51:24.999,2023-11-01,1.0,0.0,0.0,0.0,0.0,...,616.0,0.0,1.0,0.230769,14.0,10.0,10.710830,2.571197,7.259573,26.193051
2,1366776_35046_16-23_2023-11-01 00:30:59.797,1366776,35046,2023-11-01 00:30:59.797,2023-11-01,1.0,0.0,0.0,0.0,0.0,...,616.0,0.0,1.0,0.230769,14.0,10.0,10.642857,2.541195,7.261989,25.816655
3,1366776_6275451_16-23_2023-11-02 22:21:32.261,1366776,6275451,2023-11-02 22:21:32.261,2023-11-02,1.0,0.0,0.0,0.0,0.0,...,616.0,0.0,1.0,0.227848,14.0,10.0,10.555336,2.550657,7.284727,25.735649
4,1366776_78053_16-23_2023-11-02 22:21:34.799,1366776,78053,2023-11-02 22:21:34.799,2023-11-02,1.0,0.0,0.0,0.0,0.0,...,616.0,0.0,1.0,0.230769,14.0,10.0,10.766574,2.570982,7.268245,25.774037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770159,1896641_87731_16-23_2023-11-02 08:14:21.524,1896641,87731,2023-11-02 08:14:21.524,2023-11-02,0.0,0.0,0.0,0.0,42.0,...,616.0,0.0,1.0,0.230769,15.0,11.0,11.051869,2.544059,7.260547,26.050334
770160,1896641_505604_16-23_2023-11-02 08:14:24.458,1896641,505604,2023-11-02 08:14:24.458,2023-11-02,0.0,0.0,0.0,0.0,42.0,...,616.0,0.0,1.0,0.230769,14.0,10.0,11.045118,2.544860,7.299398,25.983468
770161,1896641_25212_16-23_2023-11-02 08:14:25.748,1896641,25212,2023-11-02 08:14:25.748,2023-11-02,0.0,0.0,0.0,0.0,42.0,...,616.0,0.0,1.0,0.232323,14.0,10.0,11.084913,2.547388,7.282787,25.879578
770162,1900765_95157_16-23_2023-11-02 06:08:25.900,1900765,95157,2023-11-02 06:08:25.900,2023-11-02,0.0,0.0,0.0,0.0,0.0,...,616.0,0.0,1.0,0.231884,14.0,10.0,10.706166,2.572736,7.302446,26.250963


In [36]:
df_train.to_parquet('/kaggle/working/train_features_merged.parquet', index=False)


In [31]:
df_train

,id1,id2,id3,id4,id5,f1,f2,f3,f4,f5,...,days_since_last_txn,txn_hour_std,txn_date_range,txn_hour_safe_mode,txn_dow_safe_mode,txn_dom_safe_mode,f368_safe_mode,f369_safe_mode,f374_safe_mode,id8_safe_mode
0,1362907_91950_16-23_2023-11-04 18:56:26.000794,1362907,91950,2023-11-04 18:56:26.000794,2023-11-04,0.0,0.0,0.0,0.0,0.0,...,616.0,7.254571,25.892480,0.0,0.0,2.0,PR,D,EP,59420000
1,1082599_88356_16-23_2023-11-04 06:08:53.373,1082599,88356,2023-11-04 06:08:53.373,2023-11-04,0.0,9.0,0.0,0.0,0.0,...,616.0,7.403134,27.398046,0.0,0.0,2.0,PGC,D,EP,59420000
2,1888466_958700_16-23_2023-11-05 10:07:28.000725,1888466,958700,2023-11-05 10:07:28.000725,2023-11-05,0.0,0.0,0.0,0.0,22.0,...,616.0,7.280517,26.227776,0.0,4.0,2.0,PR,D,EP,59420000
3,1888971_795739_16-23_2023-11-04 12:25:28.244,1888971,795739,2023-11-04 12:25:28.244,2023-11-04,0.0,0.0,0.0,0.0,0.0,...,616.0,7.268310,25.974987,0.0,0.0,2.0,PR,D,EP,59420000
4,1256369_82296_16-23_2023-11-05 06:45:26.657,1256369,82296,2023-11-05 06:45:26.657,2023-11-05,0.0,0.0,0.0,0.0,0.0,...,616.0,7.281326,25.956854,0.0,0.0,2.0,PR,D,EP,59420000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369296,1874443_95537_16-23_2023-11-05 09:21:24.182,1874443,95537,2023-11-05 09:21:24.182,2023-11-05,0.0,0.0,0.0,0.0,0.0,...,616.0,7.254479,25.931450,0.0,0.0,2.0,PR,D,EP,59420000
369297,1541978_5718_16-23_2023-11-05 00:56:43.946,1541978,5718,2023-11-05 00:56:43.946,2023-11-05,23.0,0.0,1.0,0.0,10.0,...,616.0,7.249058,25.909814,0.0,0.0,2.0,PR,D,EP,59420000
369298,1887841_85905_16-23_2023-11-05 20:40:43.312,1887841,85905,2023-11-05 20:40:43.312,2023-11-05,0.0,0.0,0.0,0.0,0.0,...,616.0,7.290083,25.869584,0.0,0.0,2.0,PR,D,EP,59420000
369299,1569367_944713_16-23_2023-11-05 00:43:04.335,1569367,944713,2023-11-05 00:43:04.335,2023-11-05,0.0,0.0,0.0,0.0,0.0,...,616.0,7.253589,25.781202,0.0,0.0,2.0,PR,D,EP,59420000


In [38]:
features_id3.to_parquet('/kaggle/working/trans_features_train.parquet', index=False)
